In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import DataFrame 
import nltk

from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn import neighbors
from scipy.spatial.distance import cosine
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

import re
import string
from wordcloud import WordCloud, STOPWORDS
from sklearn.metrics import mean_squared_error

In [3]:
df_meta=pd.read_csv('data/Meta_Clothing_Shoes_Jewelry_Reviews.csv')

In [6]:

if 'Unnamed: 0' in df_meta.columns:
    del df_meta['Unnamed: 0']
else:
    exit
df_meta.columns



Index(['asin', 'related', 'title', 'price', 'salesRank', 'imUrl', 'brand',
       'categories', 'description'],
      dtype='object')

In [9]:
df_meta.head(10)

,asin,related,title,price,salesRank,imUrl,brand,categories,description
0,0000037214,"{'also_viewed': ['B00JO8II76', 'B00DGN4R1Q', '...",Purple Sequin Tiny Dancer Tutu Ballet Dance Fa...,6.99,{'Clothing': 1233557},http://ecx.images-amazon.com/images/I/31mCncNu...,Big Dreams,"[['Clothing, Shoes & Jewelry', 'Girls'], ['Clo...",NaN
1,0000031887,"{'also_bought': ['0000031852', '0000031895', '...",Ballet Dress-Up Fairy Tutu,6.79,{'Sports &amp; Outdoors': 8547},http://ecx.images-amazon.com/images/I/314qZjYe...,Boutique Cutie,"[['Clothing, Shoes & Jewelry', 'Girls', 'Cloth...",This adorable basic ballerina tutu is perfect ...
2,0123456479,"{'also_bought': ['B000BMTCK6', 'B0006JCGUM', '...",SHINING IMAGE HUGE PINK LEATHER JEWELRY BOX / ...,64.98,{'Kitchen & Dining': 16987},http://ecx.images-amazon.com/images/I/413tGhqo...,NaN,"[['Clothing, Shoes & Jewelry', 'Novelty, Costu...",Elegance par excellence. Hand-crafted of the f...
3,0456844570,"{'also_viewed': ['B008MTRT1O', 'B00BUG47S4', '...",RiZ Women's Beautify Crafted &frac12; Rimmed F...,NaN,{'Clothing': 1180499},http://ecx.images-amazon.com/images/I/31QZTHxv...,NaN,"[['Clothing, Shoes & Jewelry', 'Women', 'Acces...",NaN
4,0456808574,"{'also_viewed': ['B00A6JD8EA', 'B00BFL4FTA', '...",Lantin White Visor Wrap Around Ski Style Aviat...,NaN,{'Clothing': 1038084},http://ecx.images-amazon.com/images/I/31UsrgT5...,NaN,"[['Clothing, Shoes & Jewelry', 'Women', 'Acces...",NaN
5,0456830197,{'also_bought': ['729300236X']},NVC Unisex Light Weight Silver 'Dakota' Glasse...,NaN,{'Shoes': 257607},http://ecx.images-amazon.com/images/I/21PGEX1t...,NaN,"[['Clothing, Shoes & Jewelry', 'Women', 'Acces...",NaN
6,0456856293,"{'also_viewed': ['B005P1HF2U', 'B006K5JWOE', '...",Kismeth Eyewear Classic Large Top Gun Aviator ...,NaN,{'Shoes': 399415},http://ecx.images-amazon.com/images/I/31-NheYD...,NaN,"[['Clothing, Shoes & Jewelry', 'Women', 'Acces...",NaN
7,0456840532,"{'also_viewed': ['B001K9DPPC', 'B00BV9MU2K', '...",Max-MPH Black - Large Wayfarer Sunglasses Avai...,NaN,{'Clothing': 2728771},http://ecx.images-amazon.com/images/I/11q4qGCd...,NaN,"[['Clothing, Shoes & Jewelry', 'Women', 'Acces...",NaN
8,0456787283,"{'also_viewed': ['B00AIPLH22', 'B005P1H8KO', '...",FX1 Small Adult Aviator Sunglasses with Silver...,NaN,{'Clothing': 456854},http://ecx.images-amazon.com/images/I/31dS6eGu...,NaN,"[['Clothing, Shoes & Jewelry', 'Women', 'Acces...",NaN
9,0456838384,"{'also_viewed': ['B001K9DPPC', 'B0042FHTDK', '...",Riz Small Unisex (Mens/ Womens) Black Classic ...,NaN,{'Clothing': 437391},http://ecx.images-amazon.com/images/I/31b1J21i...,NaN,"[['Clothing, Shoes & Jewelry', 'Women', 'Acces...",NaN


In [15]:
df_meta['related'][0]

"{'also_viewed': ['B00JO8II76', 'B00DGN4R1Q', 'B00E1YRI4C']}"

In [19]:
df_meta[df_meta.eq('B00BYH9DPG').any(1)]

,asin,related,title,price,salesRank,imUrl,brand,categories,description
1141128,B00BYH9DPG,"{'also_bought': ['B00ENOPRIO', 'B00DETR6YM', '...",J.TOMSON Womens Boyfriend Blazer,NaN,{'Clothing': 1950},http://ecx.images-amazon.com/images/I/41FUlaM1...,NaN,"[['Clothing, Shoes & Jewelry', 'Women', 'Cloth...",NaN


In [22]:
df_meta[df_meta.eq('B008P7L92S').any(1)]

,asin,related,title,price,salesRank,imUrl,brand,categories,description
870255,B008P7L92S,"{'also_bought': ['B00J2L4EUA', 'B00902G8RS', '...",Allegra K Women's 2-Fer Bracelet Sleeve Ruched...,NaN,{'Clothing': 3909},http://ecx.images-amazon.com/images/I/41lN%2Bk...,NaN,"[['Clothing, Shoes & Jewelry', 'Women', 'Cloth...",NaN


In [1]:
import ast

In [5]:
ast.literal_eval('['B00AM481BO' 'Top Moda Women Pack-72 Boots' 'http://ecx.images-amazon.com/images/I/41UqolGlSkL._SY395_.jpg']')

SyntaxError: invalid syntax (<ipython-input-5-b7f100cba4a2>, line 1)

In [7]:
type('['B00AM481BO' 'Top Moda Women Pack-72 Boots' 'http://ecx.images-amazon.com/images/I/41UqolGlSkL._SY395_.jpg']')

SyntaxError: invalid syntax (<ipython-input-7-2b3f82f0ebdd>, line 1)